In [ ]:
# default_exp engine

In [ ]:
#export
import chess.engine
import chess.pgn
import pandas as pd

In [ ]:
stockfish = "stockfish_14_linux_x64_popcnt/stockfish_14_x64_popcnt" # location of stockfish executable or alias

In [ ]:
engine = chess.engine.SimpleEngine.popen_uci(stockfish)

In [ ]:
pgn = open("data/magnus.pgn")

In [ ]:
game = chess.pgn.read_game(pgn)

In [ ]:
game.headers

Headers(Event='Rated Blitz game', Site='https://lichess.org/lAV0T0zl', Date='2021.12.23', Round='?', White='DrNykterstein', Black='may6enexttime', Result='1-0', BlackElo='2974', BlackRatingDiff='-2', BlackTitle='GM', ECO='B20', Termination='Normal', TimeControl='180+0', UTCDate='2021.12.23', UTCTime='23:28:07', Variant='Standard', WhiteElo='3212', WhiteRatingDiff='+2', WhiteTitle='GM')

In [ ]:
#export
def evaluate_game(game, engine, limit=chess.engine.Limit(time=.1)):
    board = game.board()
    moves = game.mainline_moves()
    evals = []
    for move in moves:
        board.push(move)
        info = engine.analyse(board, limit=limit)
        score = info["score"].pov(chess.WHITE).score()
        evals.append(score)
    return evals

In [ ]:
evals = evaluate_game(game, engine)

In [ ]:
evals

[44,
 41,
 -20,
 -8,
 -4,
 13,
 15,
 60,
 79,
 87,
 -55,
 -50,
 -64,
 -73,
 -71,
 -68,
 -112,
 -146,
 -115,
 -153,
 -161,
 -168,
 -118,
 -50,
 -49,
 -72,
 -84,
 -46,
 -61,
 -80,
 -52,
 -4,
 -103,
 -35,
 -45,
 5,
 0,
 0,
 -22,
 316,
 405,
 372,
 381,
 413,
 417,
 417,
 416,
 554,
 542,
 519,
 531,
 525,
 523,
 625,
 633]

In [ ]:
#export
def evaluate_pgn(pgn, engine, limit=chess.engine.Limit(time=.1)):
    game = chess.pgn.read_game(pgn)
    all_evals = []
    while game:
        all_evals.append(evaluate_game(game, engine, limit))
        game = chess.pgn.read_game(pgn)
    df = pd.DataFrame(all_evals)
    return df

In [ ]:
pgn = open("data/magnus.pgn")

In [ ]:
evals = evaluate_pgn(pgn, engine)

In [ ]:
evals

,0,1,2,3,4,5,6,7,8,9,...,127,128,129,130,131,132,133,134,135,136
0,34,31,-5,-12,-7,12,3,48,50,75,...,None,None,None,None,None,None,None,None,None,None
1,36,33,49,37,44,43,48,20,22,24,...,None,None,None,None,None,None,None,None,None,None
2,43,33,-16,-6,-14,-7,6,0,20,15,...,None,None,None,None,None,None,None,None,None,None
3,29,25,40,23,24,25,25,11,9,18,...,None,None,None,None,None,None,None,None,None,None
4,40,57,-21,-22,-29,-16,-5,-8,-45,-16,...,None,None,None,None,None,None,None,None,None,None


In [ ]:
evals.to_csv("data/magnus_evals.csv")